# Container Lab Guide

Welcome to the container lab! This guide will walk you through running your containerized application in four environments:
1. **Local**
2. **Azure Container Instance (ACI)**
3. **Azure Container Apps (ACA)**
4. **Kubernetes with Azure Kubernetes Service (AKS)**

Each environment will return a unique letter, and by the end, you'll collect all the letters for the final answer.


## 1. Run the Container Locally

### Steps:
1. Build the Docker image using `Dockerfile.local`.
2. Run the container locally.
3. Check the output for the letter specific to the `local` environment.


In [ ]:
# Build the Docker image for local
!docker build -t letter-local -f Dockerfile.local .

# Run the container locally
!docker run letter-local


## 2. Deploy to Azure Container Instance (ACI)

### Steps:
1. Build and push the Docker image to your Azure Container Registry (ACR).
2. Deploy the container to ACI.
3. Check the logs in Azure Portal or CLI for the letter specific to `aci`.


In [ ]:
# Build and push the Docker image for ACI
!docker build -t <your-registry-name>.azurecr.io/letter-aci:latest -f Dockerfile.aci .
!docker push <your-registry-name>.azurecr.io/letter-aci:latest

# Deploy the container to ACI
!az container create \
  --resource-group <your-resource-group> \
  --name letter-aci \
  --image <your-registry-name>.azurecr.io/letter-aci:latest \
  --registry-login-server <your-registry-name>.azurecr.io \
  --registry-username <your-registry-name> \
  --registry-password <your-acr-password> \
  --environment-variables ENVIRONMENT=aci \
  --dns-name-label <unique-dns-label> \
  --ports 5000


## 3. Deploy to Azure Container Apps (ACA)

### Steps:
1. Build and push the Docker image to ACR.
2. Deploy the container to ACA.
3. Check the logs in Azure Portal for the letter specific to `containerapps`.


In [ ]:
# Build and push the Docker image for ACA
!docker build -t <your-registry-name>.azurecr.io/letter-containerapps:latest -f Dockerfile.containerapps .
!docker push <your-registry-name>.azurecr.io/letter-containerapps:latest

# Deploy the container to ACA
!az containerapp create \
  --name letter-containerapps \
  --resource-group <your-resource-group> \
  --environment <your-containerapps-env> \
  --image <your-registry-name>.azurecr.io/letter-containerapps:latest \
  --registry-server <your-registry-name>.azurecr.io \
  --registry-username <your-registry-name> \
  --registry-password <your-acr-password> \
  --env-vars ENVIRONMENT=containerapps \
  --ingress external --target-port 5000


## 4. Deploy to Azure Kubernetes Service (AKS)

### Steps:
1. Set up an AKS cluster in Azure.
2. Deploy the container using Kubernetes deployment and service YAML.
3. Expose the application using a LoadBalancer service to retrieve the letter for `kubernetes`.


In [ ]:
# Step 1: Create an AKS cluster
!az aks create \
  --resource-group <your-resource-group> \
  --name <your-cluster-name> \
  --node-count 1 \
  --enable-addons monitoring \
  --generate-ssh-keys

In [ ]:
# Step 2: Authenticate kubectl with the AKS cluster
!az aks get-credentials \
  --resource-group <your-resource-group> \
  --name <your-cluster-name>

In [ ]:
# Step 3: Apply the deployment and service YAML
deployment_yaml = """
apiVersion: apps/v1
kind: Deployment
metadata:
  name: letter-deployment
spec:
  replicas: 1
  selector:
    matchLabels:
      app: letter-app
  template:
    metadata:
      labels:
        app: letter-app
    spec:
      containers:
      - name: letter-container
        image: <your-registry-name>.azurecr.io/letter-containerapps:latest
        ports:
        - containerPort: 5000
        env:
        - name: ENVIRONMENT
          value: "kubernetes"
"""

service_yaml = """
apiVersion: v1
kind: Service
metadata:
  name: letter-service
spec:
  type: LoadBalancer
  selector:
    app: letter-app
  ports:
    - protocol: TCP
      port: 80
      targetPort: 5000
"""

In [ ]:
# Save YAML files
with open("deployment.yaml", "w") as f:
    f.write(deployment_yaml)

with open("service.yaml", "w") as f:
    f.write(service_yaml)

In [ ]:
# Apply the YAML files to the cluster
!kubectl apply -f deployment.yaml
!kubectl apply -f service.yaml

In [ ]:
# Step 4: Get the public IP of the service
!kubectl get service letter-service